# Project for Generative AI
## Comparing effect of DAE on Noise Profiles
## Adi Ghosh and Zahra Rahmani

### Load Library files

In [ ]:
import numpy 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist, .fashion_mnist, cifar10

## MNIST

### Load Image Data

In [ ]:
(MNIST_X_train, MNIST_y_train), (MNIST_X_test, MNIST_y_test) = mnist.load_data()

### Formatting Image Data

In [ ]:
plt.subplot(221)
plt.imshow(MNIST_X_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(MNIST_X_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(MNIST_X_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(MNIST_X_train[3], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()
num_pixels = MNIST_X_train.shape[1] * MNIST_X_train.shape[2]
MNIST_X_train = MNIST_X_train.reshape(MNIST_X_train.shape[0], num_pixels).astype('float32')
MNIST_X_test = MNIST_X_test.reshape(MNIST_X_test.shape[0], num_pixels).astype('float32')
MNIST_X_train = MNIST_X_train / 255
MNIST_X_test = MNIST_X_test / 255

### Add Noise to Image

In [ ]:
noise_factor = 0.2
#Gaussian Noise
MNIST_x_train_guassian = MNIST_X_train + noise_factor * numpy.random.normal(loc=0.0, scale=1.0, size=MNIST_X_train.shape)
MNIST_x_test_guassian = MNIST_X_test + noise_factor * numpy.random.normal(loc=0.0, scale=1.0, size=MNIST_X_test.shape)
MNIST_x_train_guassian = numpy.clip(MNIST_x_train_guassian, 0., 1.)
MNIST_x_test_guassian = numpy.clip(MNIST_x_test_guassian, 0., 1.)
plt.subplot(221)
plt.imshow(MNIST_x_train_guassian[0], cmap=plt.get_cmap('gray'))
plt.subplot(222)
plt.imshow(MNIST_x_train_guassian[1], cmap=plt.get_cmap('gray'))
plt.subplot(223)
plt.imshow(MNIST_x_train_guassian[2], cmap=plt.get_cmap('gray'))
plt.subplot(224)
plt.imshow(MNIST_x_train_guassian[3], cmap=plt.get_cmap('gray'))
# show the plot
plt.show()

## Defining DAE Architecture

### Architecture 1

In [ ]:
# create model
model = Sequential()
model.add(Dense(500, input_dim=num_pixels, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(784, activation='sigmoid'))

### Architecture 2

## Network

### Training

In [ ]:
# Training model
model.fit(x_train_noisy, X_train, validation_data=(x_test_noisy, X_test), epochs=2, batch_size=200)

### Evaluation

In [ ]:
# Final evaluation of the model
pred = model.predict(x_test_noisy)
X_test = numpy.reshape(X_test, (10000,28,28)) *255
pred = numpy.reshape(pred, (10000,28,28)) *255
x_test_noisy = numpy.reshape(x_test_noisy, (-1,28,28)) *255
plt.figure(figsize=(20, 4))
print("Test Images")
for i in range(10,20,1):
    plt.subplot(2, 10, i+1)
    plt.imshow(X_test[i,:,:], cmap='gray')
    curr_lbl = y_test[i]
    plt.title("(Label: " + str(curr_lbl) + ")")
plt.show()    
plt.figure(figsize=(20, 4))
print("Test Images with Noise")
for i in range(10,20,1):
    plt.subplot(2, 10, i+1)
    plt.imshow(x_test_noisy[i,:,:], cmap='gray')
plt.show()    
plt.figure(figsize=(20, 4))
print("Reconstruction of Noisy Test Images")
for i in range(10,20,1):
    plt.subplot(2, 10, i+1)
    plt.imshow(pred[i,:,:], cmap='gray')  
plt.show()